In [1]:
import requests
import pandas as pd 
import io
import re
!pip install openpyxl
!pip install beautifulsoup4

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 26.0.1
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 26.0.1
[notice] To update, run: pip install --upgrade pip


In [16]:
import requests
from bs4 import BeautifulSoup

def get_latest_link():
    url = "https://www.ons.gov.uk/economy/economicoutputandproductivity/output/datasets/ukretailfootfall"
    response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
    soup = BeautifulSoup(response.text, "html.parser")

    # Find all links to .xlsx files
    links = soup.find_all("a", href=True)
    excel_links = [a['href'] for a in links if a['href'].endswith('.xlsx')]

    # The most recent is usually the first one
    if excel_links:
        latest_link = "https://www.ons.gov.uk" + excel_links[0]
        print("Most recent dataset link:", latest_link)
        return latest_link
    else:
        print("No Excel links found.")
        return ''
    

def get_excel(url):
    try:
        # Download into memory
        resp = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'}, timeout=60)
        resp.raise_for_status()  # raises HTTPError for 4xx/5xx

        # Optional: lightweight content-type check (won't always be accurate)
        ctype = resp.headers.get('Content-Type', '')
        if 'excel' not in ctype and 'spreadsheetml' not in ctype:
            print(f"Warning: unexpected content type: {ctype}")

        # Read the entire workbook: sheet_name=None returns a dict of DataFrames
        xls = pd.read_excel(io.BytesIO(resp.content), sheet_name=None, engine='openpyxl')

        # Output the titles (sheet names)
        print("Sheets found:", ",\n ".join(xls.keys()))

        # Example: preview each sheet
        # for name, df in xls.items():
        #     print(f"\n=== Sheet: {name} ===")
        #     print(df.head())
        
    except requests.exceptions.Timeout:
        print("Error: download timed out.")
    except requests.exceptions.HTTPError as e:
        print(f"HTTP error: {e}")
    except requests.exceptions.RequestException as e:
        print(f"Network error: {e}")
    except ValueError as e:
        # pandas read_excel errors (e.g., corrupted file)
        print(f"Excel parse error: {e}")
    return xls

def get_version(xls):
    version = ""
    try:
        raw = xls['Cover'].iloc[3,0]  # A5 -> row index 4, column 0

        m = re.search(r"(\d{1,2}\s+[A-Za-z]+\s+\d{4})", raw)
        if m:       
            version = pd.to_datetime(m.group(1), dayfirst=True).strftime('%Y-%m-%d')
        else:
            # fallback: try to coerce any parsable date inside the string
            parsed = pd.to_datetime(raw, dayfirst=True, errors='coerce')
            version = parsed.strftime('%Y-%m-%d') if not pd.isna(parsed) else ""
    except Exception as e:
        print('Could not extract publication date from Cover sheet:', e)
        version = ""
    return version

 
xls = get_excel(get_latest_link()) # display(df.head())  # or print(df.head()) if not in a notebook
version = get_version(xls)

version

Most recent dataset link: https://www.ons.gov.uk/file?uri=/economy/economicoutputandproductivity/output/datasets/ukretailfootfall/12february2026/ukretailfootfalldataset120226.xlsx
Sheets found: Cover,
 Contents,
 Notes,
 1.Weekly by region,
 2.Weekly by site type,
 3.Weekly by region and site,
 4.Monthly by region,
 5.Monthly by site type,
 6.Monthly by region and site


'2026-02-12'

In [12]:
sheets

['1.Weekly by region',
 '2.Weekly by site type',
 '3.Weekly by region and site',
 '4.Monthly by region',
 '5.Monthly by site type',
 '6.Monthly by region and site']

In [13]:


def parse_footfall_data(xls):
    sheets =list(xls.keys())[3:]
    period_type = 
    for sheet in sheets:
        df = xls.parse(sheet, header = 4).set_index(['Date'])
        df.index = pd.to_datetime(df.index)

        if 'Week' in sheet:
            period_type = 'week'
        elif 'Month' in sheet:
            period_type = 'month'

        if 'region and site' in sheet.lower():
            site_type = ...
            region ...

        else:




df.stack().head().reset_index().rename(columns = {'level_1':'region',0:'footfall_index','Date':'period_end_dt'}).assign(site_type = 'all', period_type = 'week')

,period_end_dt,region,footfall_index,site_type,period_type
0,2024-07-07,UK total,100.640079,all,week
1,2024-07-07,England,100.894023,all,week
2,2024-07-07,Northern Ireland,109.402150,all,week
3,2024-07-07,Scotland,96.063476,all,week
4,2024-07-07,Wales,99.099980,all,week


In [15]:
url = 'https://www.ons.gov.uk/file?uri=/economy/economicoutputandproductivity/output/datasets/ukretailfootfall/12february2026/ukretailfootfalldataset120226.xlsx'

parse_footfall_data(url)

HTTPError: HTTP Error 403: Forbidden

In [18]:
import re
from typing import Dict
import pandas as pd

# These are the only site-type labels present in the ONS sheets
SITE_TYPES = [
    "District or Local Centre",
    "Retail Parks",
    "Town and City Centres",
]

# ---------- Helpers ----------

def _normalize_table(df: pd.DataFrame) -> pd.DataFrame:
    """
    Given a sheet already loaded to a DataFrame (with unknown intro rows),
    find the row that contains 'Date', use that row as the header,
    and return the clean table that starts immediately below it.
    """
    raw = df.copy()

    # Locate the first occurrence of 'Date' anywhere in the sheet
    header_row = None
    header_col = None
    for i in range(raw.shape[0]):
        for j in range(raw.shape[1]):
            val = raw.iat[i, j]
            if isinstance(val, str) and val.strip() == "Date":
                header_row, header_col = i, j
                break
        if header_row is not None:
            break
    if header_row is None:
        raise ValueError("Could not find a header row containing 'Date'")

    # Trim columns left of the 'Date' cell (gets rid of notes/blank cols)
    trimmed = raw.iloc[:, header_col:].copy()

    # Promote the 'Date' row to header and drop rows above it
    trimmed.columns = trimmed.iloc[header_row].astype(str).tolist()
    trimmed = trimmed.iloc[header_row + 1 :].reset_index(drop=True)

    # Drop unnamed/empty columns and fully empty rows
    trimmed = trimmed.loc[:, ~trimmed.columns.to_series().str.contains(r"^Unnamed", na=False)]
    trimmed = trimmed.dropna(how="all")

    # Ensure the 'Date' column exists (spelling/case is as in the workbook)
    if "Date" not in trimmed.columns:
        raise ValueError("Normalized table does not contain a 'Date' column after header detection")

    return trimmed


_site_type_re = re.compile(
    r"(.*)\s+(District or Local Centre|Retail Parks|Town and City Centres)$"
)

def _melt_week_region(df: pd.DataFrame) -> pd.DataFrame:
    out = df.melt(id_vars=["Date"], var_name="region", value_name="footfall_index")
    out["site_type"] = "all"
    out["period_type"] = "week"
    return out

def _melt_week_site(df: pd.DataFrame) -> pd.DataFrame:
    out = df.melt(id_vars=["Date"], var_name="site_type", value_name="footfall_index")
    out["region"] = "UK total"
    out["period_type"] = "week"
    return out

def _melt_week_region_site(df: pd.DataFrame) -> pd.DataFrame:
    long_df = df.melt(id_vars=["Date"], var_name="combo", value_name="footfall_index")

    def split_combo(s):
        if pd.isna(s):
            return pd.Series({"region": None, "site_type": None})
        s = str(s)
        m = _site_type_re.match(s)
        if m:
            return pd.Series({"region": m.group(1).strip(), "site_type": m.group(2).strip()})
        # Safe fallback if formatting shifts slightly
        for st in SITE_TYPES:
            if s.endswith(st):
                return pd.Series({"region": s[: -len(st)].strip(), "site_type": st})
        return pd.Series({"region": s, "site_type": None})

    parts = long_df["combo"].apply(split_combo)
    long_df = pd.concat([long_df.drop(columns=["combo"]), parts], axis=1)
    long_df["period_type"] = "week"
    return long_df

def _melt_month_region(df: pd.DataFrame) -> pd.DataFrame:
    out = df.melt(id_vars=["Date"], var_name="region", value_name="footfall_index")
    out["site_type"] = "all"
    out["period_type"] = "month"
    return out

def _melt_month_site(df: pd.DataFrame) -> pd.DataFrame:
    out = df.melt(id_vars=["Date"], var_name="site_type", value_name="footfall_index")
    out["region"] = "UK total"
    out["period_type"] = "month"
    return out

def _melt_month_region_site(df: pd.DataFrame) -> pd.DataFrame:
    long_df = df.melt(id_vars=["Date"], var_name="combo", value_name="footfall_index")
    parts = long_df["combo"].apply(
        lambda s: pd.Series(_site_type_re.match(str(s)).groups(), index=["region", "site_type"])
        if _site_type_re.match(str(s))
        else pd.Series({"region": str(s), "site_type": None})
    )
    long_df = pd.concat([long_df.drop(columns=["combo"]), parts], axis=1)
    long_df["period_type"] = "month"
    return long_df

# ---------- Public function (what you’ll call) ----------

def parse_footfall_data(xls: Dict[str, pd.DataFrame], version: str = "") -> pd.DataFrame:
    """
    Parameters
    ----------
    xls : dict[str, DataFrame]
        Output of pd.read_excel(..., sheet_name=None). Keys are sheet names.
    version : str
        Version string (e.g., publication date) to stamp onto every row.

    Returns
    -------
    DataFrame with columns:
      period_start_dt, period_end_dt, period_type, region, site_type, footfall_index, version
    """

    # Helper to find the correct sheet by (partial) name, case-insensitive.
    def _pick(name_part: str) -> str:
        for k in xls.keys():
            if name_part.lower() in str(k).lower():
                return k
        raise KeyError(f"Sheet containing '{name_part}' not found in workbook")

    # 1) Normalize each sheet to get a clean table with a 'Date' header.
    wk_reg  = _normalize_table(xls[_pick("Weekly by region")])
    wk_site = _normalize_table(xls[_pick("Weekly by site type")])
    wk_r_s  = _normalize_table(xls[_pick("Weekly by region and site")])

    mo_reg  = _normalize_table(xls[_pick("Monthly by region")])
    mo_site = _normalize_table(xls[_pick("Monthly by site type")])
    mo_r_s  = _normalize_table(xls[_pick("Monthly by region and site")])

    # 2) Melt all six into long
    w_reg  = _melt_week_region(wk_reg)
    w_site = _melt_week_site(wk_site)
    w_r_s  = _melt_week_region_site(wk_r_s)

    m_reg  = _melt_month_region(mo_reg)
    m_site = _melt_month_site(mo_site)
    m_r_s  = _melt_month_region_site(mo_r_s)

    combined = pd.concat([w_reg, w_site, w_r_s, m_reg, m_site, m_r_s], ignore_index=True)

    # 3) Dates: weekly dates are END; monthly dates are START
    combined = combined.rename(columns={"Date": "date"})
    combined["date"] = pd.to_datetime(combined["date"], errors="coerce")

    combined["period_start_dt"] = combined["date"].where(
        combined["period_type"].eq("month"),
        combined["date"] - pd.to_timedelta(6, unit="D"),
    )
    combined["period_end_dt"] = combined["date"].where(
        combined["period_type"].eq("week"),
        combined["date"],
    )
    is_month = combined["period_type"].eq("month")
    combined.loc[is_month, "period_end_dt"] = combined.loc[is_month, "period_start_dt"] + pd.offsets.MonthEnd(1)

    # 4) Clean up text, types, and add version
    combined["region"] = combined["region"].astype(str).str.strip()
    combined["site_type"] = combined["site_type"].astype(str).str.strip()
    combined.loc[combined["region"].eq("nan"), "region"] = None
    combined.loc[combined["site_type"].eq("nan"), "site_type"] = None

    combined["footfall_index"] = pd.to_numeric(combined["footfall_index"], errors="coerce")
    combined["version"] = version

    out = combined[
        ["period_start_dt", "period_end_dt", "period_type", "region", "site_type", "footfall_index", "version"]
    ].copy()

    # Emit dates as date (not datetime)
    out["period_start_dt"] = pd.to_datetime(out["period_start_dt"]).dt.date
    out["period_end_dt"]   = pd.to_datetime(out["period_end_dt"]).dt.date

    return out


In [22]:
df = parse_footfall_data(xls, version = version)
df.to_csv('testdata.xlsx')

In [ ]:
df

In [ ]:
# List all sheet names in the Excel file
excel_file = pd.ExcelFile('ukretailfootfall.xlsx')
print('Available sheets:', excel_file.sheet_names)

Most recent dataset link: https://www.ons.gov.uk/file?uri=/economy/economicoutputandproductivity/output/datasets/ukretailfootfall/12february2026/ukretailfootfalldataset120226.xlsx
